Alumnos:
Mikel Alberdi,
Mayra Pullupaxi

### Proyecto práctico

Consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será _SpaceInvaders-v0_ y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, una solución óptima será alcanzada cuando el agente consiga una **media de recompensa por encima de 20 puntos en modo test**. Por ello, esta media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este proyecto práctico consta de tres partes:

   1) Implementar la red neuronal que se usará en la solución
    
   2) Implementar las distintas piezas de la solución DQN
    
   3) Justificar la respuesta en relación a los resultados obtenidos

IMPORTANTE:

- Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.

- Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).

- Tened en cuenta que las versiones de librerías recomendadas son Tensorflow==1.13.1, Keras==2.2.4 y keras-rl==0.4.2

#### Importar librerías

In [2]:
from __future__ import division

from PIL import Image
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from keras.optimizers import Adam
import keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

C:\Users\mikel\anaconda3\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  DESCRIPTOR = _descriptor.FileDescriptor(
C:\Users\mikel\anaconda3\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:36: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.FieldDescriptor(
C:\Users\mikel\anaconda3\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:29: DeprecationWarning: Call to deprecated create function Descriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _TENSORSHAPEPROTO_

#### Configuración base

In [3]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

C:\Users\mikel\anaconda3\lib\site-packages\gym\envs\registration.py:564: UserWarning: WARN: The environment SpaceInvaders-v0 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
C:\Users\mikel\anaconda3\lib\site-packages\gym\utils\passive_env_checker.py:97: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  logger.warn(


In [9]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

1) Implementación de la red neuronal

In [10]:
input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE
model = Sequential()
model.add(Permute((2, 3, 1), input_shape=input_shape))
model.add(Convolution2D(32, (8, 8), strides=(4, 4)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3), strides=(1, 1)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 permute_1 (Permute)         (None, 84, 84, 4)         0         
                                                                 
 conv2d_3 (Conv2D)           (None, 20, 20, 32)        8224      
                                                                 
 activation_5 (Activation)   (None, 20, 20, 32)        0         
                                                                 
 conv2d_4 (Conv2D)           (None, 9, 9, 64)          32832     
                                                                 
 activation_6 (Activation)   (None, 9, 9, 64)          0         
                                                                 
 conv2d_5 (Conv2D)           (None, 7, 7, 64)          36928     
                                                                 
 activation_7 (Activation)   (None, 7, 7, 64)         

2) Implementación de la solución DQN

In [21]:
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps',
                              value_max=1., value_min=.1, value_test=0.1,
                              nb_steps=2500000)
memory = SequentialMemory(limit=100000, window_length=WINDOW_LENGTH)
processor = AtariProcessor()

In [22]:
dqn = DQNAgent(model=model,
               memory=memory,
               policy=policy,
               processor=processor,
               gamma=0.99,
               nb_actions=nb_actions,
               nb_steps_warmup=50000,
               target_model_update=1000,
               train_interval=20
                )
               
dqn.compile(Adam(learning_rate=0.00025),metrics=['mae'])

In [23]:
weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
checkpoint_weights_filename = 'dqn_' + env_name + '_weights_{step}.h5f'
log_filename = 'dqn_{}_log.json'.format(env_name)
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=250000)]
callbacks += [FileLogger(log_filename, interval=100)]

dqn.fit(env, callbacks=callbacks, nb_steps=3750000,verbose=2)

dqn.save_weights(weights_filename, overwrite=True)

Training for 3750000 steps ...


C:\Users\mikel\anaconda3\lib\site-packages\keras\engine\training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


     780/3750000: episode: 1, duration: 2.633s, episode steps: 780, steps per second: 296, episode reward: 10.000, mean reward:  0.013 [ 0.000,  1.000], mean action: 2.536 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
    1291/3750000: episode: 2, duration: 1.766s, episode steps: 511, steps per second: 289, episode reward:  3.000, mean reward:  0.006 [ 0.000,  1.000], mean action: 2.479 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
    1971/3750000: episode: 3, duration: 2.227s, episode steps: 680, steps per second: 305, episode reward: 14.000, mean reward:  0.021 [ 0.000,  1.000], mean action: 2.497 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
    2665/3750000: episode: 4, duration: 2.270s, episode steps: 694, steps per second: 306, episode reward:  8.000, mean reward:  0.012 [ 0.000,  1.000], mean action: 2.520 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
    3291/3750000: episode: 5, duration: 1.936s, episode steps: 626, 

C:\Users\mikel\anaconda3\lib\site-packages\keras\engine\training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


   50111/3750000: episode: 75, duration: 2.967s, episode steps: 592, steps per second: 200, episode reward:  9.000, mean reward:  0.015 [ 0.000,  1.000], mean action: 2.566 [0.000, 5.000],  loss: 0.009140, mae: 3.481089, mean_q: 4.196437, mean_eps: 0.981978
   51010/3750000: episode: 76, duration: 6.693s, episode steps: 899, steps per second: 134, episode reward: 15.000, mean reward:  0.017 [ 0.000,  1.000], mean action: 2.506 [0.000, 5.000],  loss: 0.016818, mae: 3.412151, mean_q: 4.109675, mean_eps: 0.981798
   51409/3750000: episode: 77, duration: 3.003s, episode steps: 399, steps per second: 133, episode reward:  8.000, mean reward:  0.020 [ 0.000,  1.000], mean action: 2.526 [0.000, 5.000],  loss: 0.008783, mae: 3.354248, mean_q: 4.036198, mean_eps: 0.981564
   52076/3750000: episode: 78, duration: 4.957s, episode steps: 667, steps per second: 135, episode reward: 10.000, mean reward:  0.015 [ 0.000,  1.000], mean action: 2.658 [0.000, 5.000],  loss: 0.009815, mae: 3.384423, mean_

In [24]:
# Testing part to calculate the mean reward
weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: 27.000, steps: 829
Episode 2: reward: 21.000, steps: 732
Episode 3: reward: 9.000, steps: 383
Episode 4: reward: 18.000, steps: 625
Episode 5: reward: 30.000, steps: 1183
Episode 6: reward: 22.000, steps: 810
Episode 7: reward: 21.000, steps: 807
Episode 8: reward: 20.000, steps: 768
Episode 9: reward: 32.000, steps: 1239
Episode 10: reward: 18.000, steps: 790


3) Justificación de los parámetros seleccionados y de los resultados obtenidos

En este experimento se ha entrenado el modelo durante 3.750.000 steps, de los cuales 2.500.000 han correspondido a la fase de exploración bajando epsilon desde 1 hasta 0,1. El resto de steps corresponden a la fase de explotación. El modelo se actualiza cada 1.000 steps ya que en el caso de ser con un número mayor de steps la recompensa por episodio es menor, siendo los 50.000 primeros de calentamiento. Se utiliza una memoria de 100.000 steps. Por último el factor gamma se establece en 0,99.

En cuanto a la red neuronal, se utiliza una típica para los problemas de atary, con tres capas convolucionales con activación 'relu', después una capa Flatten y dos Dense, la última con activación linear.

En el test del experimento se obtiene una media por encima de 20, siendo 7 test superiores y 3 inferiores. La puntuación máxima en el test es 32 y la mínima 9

